In [1]:
import re
import string
import mlflow
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

/home/mo-shwa/anaconda3/envs/MLOps-env/lib/python3.11/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
df = pd.read_csv('IMDB.csv')

df = df.sample(500)

df.to_csv('data.csv', index=False)

df.head()

,review,sentiment
190,I almost made a fool of myself when I was goin...,negative
450,"""Landscape after a battle"" opens with escaping...",positive
23,Most italian horror lovers seem to hate this m...,positive
585,Quentin Tarantino's partner in crime Roger Ava...,negative
508,Incarcerated train robber near Yuma breaks fre...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)

df.head()

,review,sentiment
190,almost made fool going start review saying mov...,negative
450,landscape battle open escaping prisoner snowy ...,positive
23,italian horror lover seem hate movie since con...,positive
585,quentin tarantino s partner crime roger avary ...,negative
508,incarcerated train robber near yuma break free...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    263
positive    237
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])

df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})

df.head()

,review,sentiment
190,almost made fool going start review saying mov...,0
450,landscape battle open escaping prisoner snowy ...,1
23,italian horror lover seem hate movie since con...,1
585,quentin tarantino s partner crime roger avary ...,0
508,incarcerated train robber near yuma break free...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)

X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/ShwaTech/MLOps-End-To-End-Movie-Review-Sentiment-Analysis-Project.mlflow')
dagshub.init(repo_owner='ShwaTech', repo_name='MLOps-End-To-End-Movie-Review-Sentiment-Analysis-Project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

Accessing as ShwaTech

Initialized MLflow to track repo "ShwaTech/MLOps-End-To-End-Movie-Review-Sentiment-Analysis-Project"

Repository ShwaTech/MLOps-End-To-End-Movie-Review-Sentiment-Analysis-Project initialized!

<Experiment: artifact_location='mlflow-artifacts:/42d7ddac9c90425e916f012f1393c3af', creation_time=1760644527035, experiment_id='0', last_update_time=1760644527035, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [12]:
import mlflow
import logging
import time
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "logistic_regression_model", input_example=X_test[:5])

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-10-16 23:58:26,052 - INFO - Starting MLflow run...


2025-10-16 23:58:26,724 - INFO - Logging preprocessing parameters...
2025-10-16 23:58:28,085 - INFO - Initializing Logistic Regression model...
2025-10-16 23:58:28,089 - INFO - Fitting the model...
2025-10-16 23:58:28,157 - INFO - Model training complete.
2025-10-16 23:58:28,158 - INFO - Logging model parameters...
2025-10-16 23:58:28,643 - INFO - Making predictions...
2025-10-16 23:58:28,644 - INFO - Calculating evaluation metrics...
2025-10-16 23:58:28,660 - INFO - Logging evaluation metrics...
2025-10-16 23:58:30,246 - INFO - Saving and logging the model...
2025-10-16 23:58:40,164 - INFO - Model training and logging completed in 13.44 seconds.
2025-10-16 23:58:40,165 - INFO - Accuracy: 0.656
2025-10-16 23:58:40,166 - INFO - Precision: 0.6140350877192983
2025-10-16 23:58:40,168 - INFO - Recall: 0.625
2025-10-16 23:58:40,169 - INFO - F1 Score: 0.6194690265486725
